In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
features = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/features.txt", sep='\s+', header=None, names=['column_index', 'column_name'])
features_cc = features.groupby('column_name').cumcount()
features_cc = pd.DataFrame(features_cc)
features_cc.columns = ['cumcount']

In [9]:
features_df = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/features.txt", sep='\s+', header=None, names=['column_index', 'column_name'])

# 피쳐이름에 그룹바이와 cumcount를 적용한 데이터프레임을 만든다
features_cc = features_df.groupby('column_name').cumcount() # (561, )의 시리즈 생성됨
features_cc = pd.DataFrame(features_cc) # (561, 1)의 데이터프레임으로 변환
features_cc.columns = ['cumcount'] # 칼럼명 추가
features_cc = features_cc.reset_index() # (561,2)가 된다.
features_df = features_df.reset_index() # (561,3)이 된다.

# 양쪽 데이터프레임 reset_index()의 결과로 생긴 'index'열을 기준으로 outer join(병합)한다.
# 그럼 결과적으로 index, column_index, column_name, cumcount 4개의 열을 가진 데이터프레임이 생성된다.
new_df = pd.merge(features_cc, features_df, on='index', how='outer')

# 병합에 사용되었던 index 칼럼을 드랍한다.
new_df = new_df.drop(['index'], axis=1) # column_index, column_name, cumcount의 (561,3)이 된다.

# column_name과 cumcount를 합쳐서 하나의 column_name으로 만드는 과정이다
# cumcount가 1이상일경우 column_name 뒤에 _1 또는 _2를 붙인다.
new_df['column_name'] = new_df[['column_name', 'cumcount']].apply(lambda x: x[0]+'_'+str(x[1])
                                                                if x[1]>0 else x[0], axis=1)

# cumcount를 column_name을 새로짓는 데 사용하였으므로 이제 드랍한다.
# 이 작업을 마치면 cumcount 칼럼이 없어져서 (561,2) 데이터프레임이 된다.
new_df = new_df.drop(['cumcount'], axis=1) # column_index, column_name 두개의 (561,2) 가 된다.

h = new_df['column_name'].value_counts()

for i in h:
    if (i == 3):
        print(False)

In [10]:
X_train = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+", names=new_df['column_name'].values)
y_train = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/train/y_train.txt", header=None, sep="\s+", names=['activity'])
X_test = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/test/X_test.txt", header=None, sep="\s+", names=new_df['column_name'].values)
y_test = pd.read_csv("/Users/kwonheejin/Documents/DL/dataset/HAR/UCI HAR Dataset/test/y_test.txt", header=None, sep="\s+", names=['activity'])

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7352, 561), (2947, 561), (7352, 1), (2947, 1))

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

scaled_X_train = pd.DataFrame(data=X_train, columns=new_df['column_name'].values)
scaled_X_test = pd.DataFrame(data=X_test, columns=new_df['column_name'].values)
scaled_X_train['label'] = y_train.values
scaled_X_test['label'] = y_test.values

scaled_X_train.shape, scaled_X_test.shape

((7352, 562), (2947, 562))

In [12]:
X_train_static = scaled_X_train[scaled_X_train['label'] > 3]
X_train_dynamic = scaled_X_train[scaled_X_train['label'] <= 3]
X_test_static = scaled_X_test[scaled_X_test['label'] > 3]
X_test_dynamic = scaled_X_test[scaled_X_test['label'] <= 3]

X_train_combined = scaled_X_train
X_test_combined = scaled_X_test

X_train_static.shape, X_train_dynamic.shape, X_test_static.shape, X_test_dynamic.shape, X_train_combined.shape, X_test_combined.shape

((4067, 562), (3285, 562), (1560, 562), (1387, 562), (7352, 562), (2947, 562))

In [13]:
y_train_static = X_train_static['label']
y_train_dynamic = X_train_dynamic['label']
y_train_combined = X_train_combined['label']
y_test_static = X_test_static['label']
y_test_dynamic = X_test_dynamic['label']
y_test_combined = X_test_combined['label']

X_train_static = X_train_static.drop('label', axis=1)
X_train_dynamic = X_train_dynamic.drop('label', axis=1)
X_train_combined = X_train_combined.drop('label', axis=1)
X_test_static = X_test_static.drop('label', axis=1)
X_test_dynamic = X_test_dynamic.drop('label', axis=1)
X_test_combined = X_test_combined.drop('label', axis=1)

In [15]:
X_train_static.shape, X_test_static.shape, X_train_dynamic.shape, X_test_dynamic.shape, X_train_combined.shape, X_test_combined.shape

((4067, 561), (1560, 561), (3285, 561), (1387, 561), (7352, 561), (2947, 561))

In [16]:
from scipy import stats

def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 10
STEP = 1

In [17]:
X_train_static, y_train_static = create_dataset(X_train_static, y_train_static, TIME_STEPS, STEP)
X_train_dynamic, y_train_dynamic = create_dataset(X_train_dynamic, y_train_dynamic, TIME_STEPS, STEP)
X_train_combined, y_train_combined = create_dataset(X_train_combined, y_train_combined, TIME_STEPS, STEP)
X_test_static, y_test_static = create_dataset(X_test_static, y_test_static, TIME_STEPS, STEP)
X_test_dynamic, y_test_dynamic = create_dataset(X_test_dynamic, y_test_dynamic, TIME_STEPS, STEP)
X_test_combined, y_test_combined = create_dataset(X_test_combined, y_test_combined, TIME_STEPS, STEP)

X_train_static.shape, y_train_static.shape,

/var/folders/_y/cmlp5dyd3tdb35r5_kr1z35w0000gn/T/ipykernel_9717/1253865503.py:9: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ys.append(stats.mode(labels)[0][0])


((4057, 10, 561), (4057, 1))

In [18]:
from sklearn.preprocessing import OneHotEncoder

enc_static = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc_static = enc_static.fit(y_train_static)

enc_dynamic = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc_dynamic = enc_dynamic.fit(y_train_dynamic)

enc_combined = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc_combined = enc_combined.fit(y_train_combined)

y_train_static = enc_static.transform(y_train_static)
y_test_static = enc_static.transform(y_test_static)

y_train_dynamic = enc_dynamic.transform(y_train_dynamic)
y_test_dynamic = enc_dynamic.transform(y_test_dynamic)

y_train_combined = enc_combined.transform(y_train_combined)
y_test_combined = enc_combined.transform(y_test_combined)

/Users/kwonheejin/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/kwonheejin/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/kwonheejin/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
y_train_dynamic.shape

(3275, 3)

In [23]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

def model_CNN(X_train_mod, y_train_mod):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_mod.shape[1], X_train_mod.shape[2])),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(y_train_mod.shape[1], activation='softmax')
    ])

    return model

In [27]:
model_static = model_CNN(X_train_static, y_train_static)
model_static.summary()

model_static.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 8, 32)             53888     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 4, 32)             0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 3)                 387       
                                                      

In [29]:
model_dynamic = model_CNN(X_train_dynamic, y_train_dynamic)
model_dynamic.summary()

model_dynamic.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 8, 32)             53888     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 4, 32)             0         
 g1D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 3)                 387       
                                                      

In [30]:
model_combined = model_CNN(X_train_combined, y_train_combined)
model_combined.summary()

model_combined.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 8, 32)             53888     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 4, 32)             0         
 g1D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 6)                 774       
                                                      

In [34]:
history_static = model_static.fit(X_train_static, y_train_static, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100
89/89 [==============================] - 0s 3ms/step - loss: 1.7579e-04 - acc: 1.0000 - val_loss: 0.3464 - val_acc: 0.9557
Epoch 2/100
89/89 [==============================] - 0s 3ms/step - loss: 5.9680e-06 - acc: 1.0000 - val_loss: 0.3467 - val_acc: 0.9557
Epoch 3/100
89/89 [==============================] - 0s 3ms/step - loss: 7.1611e-04 - acc: 0.9996 - val_loss: 0.6206 - val_acc: 0.9401
Epoch 4/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0119 - acc: 0.9961 - val_loss: 0.3492 - val_acc: 0.9557
Epoch 5/100
89/89 [==============================] - 0s 3ms/step - loss: 6.8797e-06 - acc: 1.0000 - val_loss: 0.3272 - val_acc: 0.9532
Epoch 6/100
89/89 [==============================] - 0s 3ms/step - loss: 7.8688e-05 - acc: 1.0000 - val_loss: 0.3239 - val_acc: 0.9540
Epoch 7/100
89/89 [==============================] - 0s 3ms/step - loss: 2.0748e-05 - acc: 1.0000 - val_loss: 0.3248 - val_acc: 0.9540
Epoch 8/100
89/89 [==============================] - 0s 3ms

89/89 [==============================] - 0s 3ms/step - loss: 0.0049 - acc: 0.9986 - val_loss: 0.7153 - val_acc: 0.9302
Epoch 62/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0704 - acc: 0.9908 - val_loss: 1.9363 - val_acc: 0.8801
Epoch 63/100
89/89 [==============================] - 0s 3ms/step - loss: 0.0309 - acc: 0.9947 - val_loss: 0.3616 - val_acc: 0.9573
Epoch 64/100
89/89 [==============================] - 0s 3ms/step - loss: 4.8697e-04 - acc: 0.9996 - val_loss: 0.2421 - val_acc: 0.9631
Epoch 65/100
89/89 [==============================] - 0s 3ms/step - loss: 3.3784e-05 - acc: 1.0000 - val_loss: 0.2541 - val_acc: 0.9614
Epoch 66/100
89/89 [==============================] - 0s 4ms/step - loss: 3.5779e-06 - acc: 1.0000 - val_loss: 0.2554 - val_acc: 0.9614
Epoch 67/100
89/89 [==============================] - 0s 3ms/step - loss: 2.3222e-05 - acc: 1.0000 - val_loss: 0.2676 - val_acc: 0.9606
Epoch 68/100
89/89 [==============================] - 0s 3ms/step - loss:

In [38]:
history_dynamic = model_dynamic.fit(X_train_dynamic, y_train_dynamic, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100
72/72 [==============================] - 1s 3ms/step - loss: 0.4029 - acc: 0.8434 - val_loss: 0.2049 - val_acc: 0.9217
Epoch 2/100
72/72 [==============================] - 0s 2ms/step - loss: 0.1321 - acc: 0.9520 - val_loss: 0.1829 - val_acc: 0.9278
Epoch 3/100
72/72 [==============================] - 0s 2ms/step - loss: 0.0530 - acc: 0.9808 - val_loss: 0.2565 - val_acc: 0.9318
Epoch 4/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0382 - acc: 0.9860 - val_loss: 0.1401 - val_acc: 0.9471
Epoch 5/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0540 - acc: 0.9847 - val_loss: 0.1525 - val_acc: 0.9471
Epoch 6/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0300 - acc: 0.9921 - val_loss: 0.1780 - val_acc: 0.9502
Epoch 7/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0106 - acc: 0.9965 - val_loss: 0.2698 - val_acc: 0.9471
Epoch 8/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0097 - a

72/72 [==============================] - 0s 3ms/step - loss: 2.2382e-05 - acc: 1.0000 - val_loss: 0.1257 - val_acc: 0.9756
Epoch 63/100
72/72 [==============================] - 0s 3ms/step - loss: 2.4956e-05 - acc: 1.0000 - val_loss: 0.1355 - val_acc: 0.9725
Epoch 64/100
72/72 [==============================] - 0s 3ms/step - loss: 3.1915e-05 - acc: 1.0000 - val_loss: 0.1424 - val_acc: 0.9736
Epoch 65/100
72/72 [==============================] - 0s 3ms/step - loss: 8.9911e-06 - acc: 1.0000 - val_loss: 0.1426 - val_acc: 0.9725
Epoch 66/100
72/72 [==============================] - 0s 3ms/step - loss: 1.2916e-04 - acc: 1.0000 - val_loss: 0.0822 - val_acc: 0.9807
Epoch 67/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0462 - acc: 0.9917 - val_loss: 0.4616 - val_acc: 0.9461
Epoch 68/100
72/72 [==============================] - 0s 3ms/step - loss: 0.0250 - acc: 0.9926 - val_loss: 0.3963 - val_acc: 0.9532
Epoch 69/100
72/72 [==============================] - 0s 3ms/step - l

In [41]:
history_combined = model_combined.fit(X_train_combined, y_train_combined, epochs=100, batch_size=32, validation_split=0.3)

Epoch 1/100
161/161 [==============================] - 1s 3ms/step - loss: 0.5016 - acc: 0.8186 - val_loss: 0.2550 - val_acc: 0.9119
Epoch 2/100
161/161 [==============================] - 0s 2ms/step - loss: 0.1048 - acc: 0.9628 - val_loss: 0.2958 - val_acc: 0.9242
Epoch 3/100
161/161 [==============================] - 0s 2ms/step - loss: 0.0522 - acc: 0.9803 - val_loss: 0.2666 - val_acc: 0.9319
Epoch 4/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0478 - acc: 0.9852 - val_loss: 0.3422 - val_acc: 0.9251
Epoch 5/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0329 - acc: 0.9868 - val_loss: 0.3524 - val_acc: 0.9237
Epoch 6/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0325 - acc: 0.9866 - val_loss: 0.2330 - val_acc: 0.9510
Epoch 7/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0079 - acc: 0.9979 - val_loss: 0.2438 - val_acc: 0.9469
Epoch 8/100
161/161 [==============================] - 0s 3ms/step - 

Epoch 62/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0075 - acc: 0.9981 - val_loss: 0.7130 - val_acc: 0.9523
Epoch 63/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0088 - acc: 0.9992 - val_loss: 0.6215 - val_acc: 0.9560
Epoch 64/100
161/161 [==============================] - 0s 3ms/step - loss: 8.3476e-06 - acc: 1.0000 - val_loss: 0.6023 - val_acc: 0.9560
Epoch 65/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0035 - acc: 0.9996 - val_loss: 0.6032 - val_acc: 0.9555
Epoch 66/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0093 - acc: 0.9973 - val_loss: 1.1998 - val_acc: 0.9192
Epoch 67/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0263 - acc: 0.9955 - val_loss: 0.7207 - val_acc: 0.9442
Epoch 68/100
161/161 [==============================] - 0s 3ms/step - loss: 0.0126 - acc: 0.9975 - val_loss: 0.4930 - val_acc: 0.9523
Epoch 69/100
161/161 [==============================] - 0s